# Introduction

This notebook aims to perform the Feature Engineering on famous IPL Datasets. After performing the steps described in this notebook, I have derived new datasets which can be found [here](https://www.kaggle.com/harshdhamecha/ipl-datasets-20082020).

# Importing Modules

In [ ]:
import pandas as pd
import numpy as np

# Matches Dataset
## Loading the Datasets

In [ ]:
matches = pd.read_csv('../input/ipl-complete-dataset-20082020/IPL Matches 2008-2020.csv', index_col='id')
matches.head()

## Data Preprocessing

In [ ]:
matches.info()

Inference
- As expected, most of the columns like `venue`, `team1`, `team2`, etc. are not having any null values.
- Let us go through all the columns one by one.


### `city`

In [ ]:
matches.loc[matches.city.isnull()]

Inference
- If you closely observe the `city` and `venue` column, you would get to know that the venue is either `Dubai International Cricket Stadium` or `Dubai International Cricket Stadium` for missing values of `city`.

We can easily fix this by putting `Dubai` and `Sharjah` as `city` where the `venue` is `Dubai International Cricket Stadium` and `Dubai International Cricket Stadium` respectively.  

We will do that but let us also first see all the unique values of `city` column to check data integrity.

In [ ]:
matches.city.unique()

Inference
- Two different values found for the same city: `Bangalore` and `Bengaluru`.  

Let us fix this and null values scenario as we discussed above. Following is the code for the same.

In [ ]:
def manage_city(matches):
    matches['city'].replace('Bengaluru', 'Bangalore', inplace=True)
    for idx in matches[matches['city'].isna()].index:
        matches.loc[idx, 'city'] = 'Sharjah' if matches.loc[idx, 'venue'] == 'Sharjah Cricket Stadium' else 'Dubai'

In [ ]:
manage_city(matches)
matches.loc[matches.city.isnull()]

In [ ]:
matches.city.unique()

We have handled the null values of `city` and replace the city name.  

We can verify the newly inserted `city`'s values by executing the following code

In [ ]:
matches.loc[[729287, 729311]]     # id's where city values were missing.

### `venue`
Thre's no any missing values in this column but it requires a few modifications.

In [ ]:
matches.venue.unique()

- Few of the stadium names has been changed now. So we need to handle this scenario.
- Two similar values: `M.Chinnaswamy Stadium` and `M Chinnaswamy Stadium`
- Some of the stadium names include city name too. We need to remove the city name from that.

In [ ]:
def change_venue(matches, venues):
    matches['venue'].replace(venues, inplace=True)

In [ ]:
# Venues which needs to be updated

venues = {'M Chinnaswamy Stadium': 'M.Chinnaswamy Stadium', 'Punjab Cricket Association Stadium, Mohali': 'Punjab Cricket Association IS Bindra Stadium',
        'Punjab Cricket Association IS Bindra Stadium, Mohali': 'Punjab Cricket Association IS Bindra Stadium', 'New Wanderers Stadium': 'The Wanderers Stadium', 
        'Rajiv Gandhi International Stadium, Uppal': 'Rajiv Gandhi International Stadium', 'MA Chidambaram Stadium, Chepauk': 'MA Chidambaram Stadium', 
        'Newlands': 'Newlands Cricket Ground', 'Kingsmead': 'Hollywoodbets Kingsmead Stadium', 'OUTsurance Oval': 'Mangaung Oval', 'Sardar Patel Stadium, Motera': 'Narendra Modi Stadium', 
        'Feroz Shah Kotla': 'Arun Jaitley Stadium', 'Vidarbha Cricket Association Stadium, Jamtha': 'Vidarbha Cricket Association Stadium', 
        'Subrata Roy Sahara Stadium': 'Maharashtra Cricket Association Stadium', 'Green Park': 'Green Park Stadium', 'Nehru Stadium': 'Jawaharlal Nehru Stadium'}

In [ ]:
change_venue(matches, venues)
matches.venue.unique()

It's done now!

### `team1`, `team2`, `toss_winner`, `winner`

In [ ]:
matches.team1.unique()

- The official name was `Rising Pune Supergiant`. So we'll replace `Rising Pune Supergiants` by `Rising Pune Supergiant`.
- Earlier known as `Delhi Daredevils` has now become the `Delhi Capitals`. We'll replace `Delhi Daredevils` by `Delhi Capitalas`.
- The official name was `Pune Warriors India` and not `Pune Warriors`. We'll handle this.

In [ ]:
def replace_team_names(matches, columns_to_modify):
    matches[columns_to_modify] = matches[columns_to_modify].replace({'Rising Pune Supergiants': 'Rising Pune Supergiant', 'Pune Warriors': 'Pune Warriors India', 
                                                               'Delhi Daredevils': 'Delhi Capitals'})

In [ ]:
columns_to_modify = ['team1', 'team2', 'toss_winner', 'winner']
replace_team_names(matches, columns_to_modify)
matches.team1.unique()

So, we have handled the team names.

### `player_of_match`, `winner`, `result` and `eliminator`

In [ ]:
matches.loc[matches.player_of_match.isnull()]

All these columns are having 4 missing values and in the same record (match). All these 4 matches were cancelled due to rain.  

Thus, we will keep `player_of_match`'s missing values **as it is** and replace null values with `no result` in ohter 3 columns.

In [ ]:
def manage_no_result(matches):
    for idx in matches[matches['result'].isna()].index:
        matches.loc[idx, 'winner'] = 'no result'   
        matches.loc[idx, 'result'] = 'no result'
        matches.loc[idx, 'result_margin'] = 'no result'

In [ ]:
manage_no_result(matches)
matches.loc[matches.player_of_match.isnull()]

### `method`

In [ ]:
matches.loc[-matches.method.isnull()]

Inference
- The column `method` has only 19 non-null values, i.e. `D/L`. It denotes the matches where D/L method is applied. 

We do not need to apply any preprocessing technique for this column.

### `neutral_venue` and Feature Engineering

Now comes the **most important** part of this notebook.

In [ ]:
matches.loc[matches.date.str.startswith('2020')]

For all the matches of 2020 season, `neutral_venue==0` which is not true as the season was scheduled in the UAE. We will change this to 1.

In [ ]:
# Manually gone through different seasons of IPL and found out the following ids which represent the playoff matches.

playoff_ids = [336038, 336039, 336040, 392237, 392238, 392239, 419162, 419163, 419165, 501268, 501269, 501270, 501271, 548378, 548379, 548380, 548381, 598070, 598071, 598072, 
               598073, 734043, 734045, 734047, 734049, 829817, 829819, 829821, 829823, 981013, 981015, 981017, 981019, 1082647, 1082648, 1082649, 1082650, 1136617, 
               1136618, 1136619, 1136620, 1181764, 1181766, 1181767, 1181768, 1237177, 1237178, 1237180, 1237181]

In [ ]:
matches.loc[playoff_ids]

**For all the playoff matches except the 2nd season(2009), `neutral_venue==0` in our Datasets which is the biggest mistake in the dataset**. We'll handle this soon.   

Now, refer [this wikipedia page](https://en.wikipedia.org/wiki/List_of_Indian_Premier_League_venues) to get a list of home venues yearwise. Following variables will help us for preprocessing and feature engineering.

In [ ]:
# All the following variables are derived manually.

# ids of all qualifier matches
qualifier_ids = [336038, 336039, 392237, 392238, 419162, 419163, 501268, 501270, 548378, 548380, 598070, 598072, 734043, 734047, 
                 829817, 829821, 981013, 981017, 1082647, 1082649, 1136617, 1136619, 1181764, 1181767, 1237177, 1237180]

# ids of all eliminator matches
eliminator_ids = [501269, 548379, 598071, 734045, 829819, 981015, 1082648, 1136618, 1181766, 1237178]

# ids of all final matches
final_ids = [336040, 392239, 419165, 501271, 548381, 598073, 734049, 829823, 981019, 1082650, 1136620, 1181768, 1237181]

# ids of all playoff matches when matches played in team2's home ground
team2_home_ids = [419163, 548380, 548381, 734043, 829817, 981013, 1181764]

# ids of all playoff matches when matches played at neutral venues 
change_neutral_venue_ids = [336038, 336039, 336040, 419162, 419165, 501268, 501270, 548378, 548379, 598070, 598071, 598072, 598073, 734045, 
                    734047, 734049, 829819, 829821, 829823, 981015, 981017, 1082648, 1082649, 1082650, 1136617, 1136620, 1181766, 1181767, 1181768]

As the value of `neutral_venue` is 0 in all the playoff matches, we need to change that to 1. we will use the `change_neutral_venue_ids` list to update the values.  
Let's do this and also replace the `neutral_venue`'s values in 2020 matches by 1 as discussed above.

In [ ]:
def change_neutral_venue(matches, change_neutral_venue_ids):
    for idx in matches.index:
        if idx in change_neutral_venue_ids:
            matches.loc[idx, 'neutral_venue'] = 1 
        if matches.loc[idx, 'date'][0:4] == '2020':
            matches.loc[idx, 'neutral_venue'] = 1

In [ ]:
change_neutral_venue(matches, change_neutral_venue_ids)
matches.loc[playoff_ids]

Now, you can see that the `neutral_venue`'s values are perfectly matching with the team. We have solved the biggest problem of the dataset. 


Players' and Teams' performance varies in Playoff Matches as compared to the League Stage Matches. Thus it becomes really necessary to analyze it. So, we are going to define some new features which will help us doing the same.

In [ ]:
def add_few_columns(eliminator_ids, qualifier_ids, final_ids, playoff_ids, team2_home_ids):
    matches['is_playoff'] = [1 if idx in playoff_ids else 0 for idx in matches.index]
    matches['is_eliminator'] = [1 if idx in eliminator_ids else 0 for idx in matches.index]
    matches['is_qualifier'] = [1 if idx in qualifier_ids else 0 for idx in matches.index]
    matches['is_final'] = [1 if idx in final_ids else 0 for idx in matches.index]
    for idx in matches.index:
        if matches.loc[idx, 'neutral_venue'] == 0:
            # Since team names has been arranged in the order (For example, MI in team1 column and CSK in team2 column if the match is schedule in MI's home) 
            # except the playoff matches. We only need to take care of the playoff matches. 
            # We will use the variable team2_home_ids which we have defined earlier to do the same.

            matches.loc[idx, 'home_of'] = matches.loc[idx, 'team2'] if idx in team2_home_ids else matches.loc[idx, 'team1']
            # Assign the value of team2 column to the newly created column home_of if the id of particular record belongs to team2_home_ids and 
            #            value of team1 column  "  "    "     "        "      "    otherwise

In [ ]:
add_few_columns(eliminator_ids, qualifier_ids, final_ids, playoff_ids, team2_home_ids)
matches.head(10)

Now, We do not required the column `eliminitaor`. Thus, let us drop that.

In [ ]:
matches.drop('eliminator', axis=1, inplace=True)
matches.columns

The players' and umpires' name format is not readable and interpretable. Names like `YK Pathan`, `KC Sangakkara`, `SR Watson`, etc. are hard to interpret. 
Thus, let us make it more interpretable. 

We need to define some variables for the same. I have used [this gist](https://gist.github.com/harshdhamecha/19100ffedaf096b8d779e4daa5e8c0e7) to define the following variables.

In [ ]:
umpire_names = {'Asad Rauf': 'Asad Rauf', 'MR Benson': 'Mark Benson', 'Aleem Dar': 'Aleem Dar', 'SJ Davis': 'Steve Davis', 'BF Bowden': 'Billy Bowden', 
                         'IL Howell': 'Ian Howell', 'DJ Harper': 'Daryl Harper', 'RE Koertzen': 'Rudi Koertzen', 'BR Doctrove': 'Billy Doctrove', 'AV Jayaprakash': 'Arani Jayaprakash', 
                         'BG Jerling': 'Brian Jerling', 'M Erasmus': 'Marais Erasmus', 'HDPK Dharmasena': 'Kumar Dharmasena', 'S Asnani': 'Sudhir Asnani', 
                         'GAV Baxter': 'Gary Baxter', 'SS Hazare': 'Sanjay Hazare', 'K Hariharan': 'Krishna Hariharan', 'SL Shastri': 'Suresh Shastri', 'SK Tarapore': 'Shavir Tarapore', 
                         'S Ravi': 'Sundaram Ravi', 'SJA Taufel': 'Simon Taufel', 'S Das': 'Subrat Das', 'PR Reiffel': 'Paul Reiffel', 'JD Cloete': 'Johan Cloete', 'AK Chaudhary': 'Anil Chaudhary', 
                         'VA Kulkarni': 'Vineet Kulkarni', 'BNJ Oxenford': 'Bruce Oxenford', 'CK Nandan': 'C. K. Nandan', 'C Shamshuddin': 'Chettithody Shamshuddin', 'NJ Llong': 'Nigel Llong', 
                         'RK Illingworth': 'Richard Illingworth', 'K Srinath': 'Krishnaraj Srinath', 'SD Fry': 'Simon Fry', 'CB Gaffaney': 'Chris Gaffaney', 'PG Pathak': 'Pashchim Pathak', 'Nitin Menon': 'Nitin Menon', 
                         'AY Dandekar': 'Anil Dandekar', 'KN Ananthapadmanabhan': 'K. N. Ananthapadmanabhan', 'A Nand Kishore': 'Nand Kishore', 'A Deshmukh': 'Abhijit Deshmukh', 'RJ Tucker': 'Rod Tucker', 'VK Sharma': 'Virender Sharma', 
                         'IJ Gould': 'Ian Gould', 'GA Pratapkumar': 'G. A. Pratapkumar', 'RB Tiffin': 'Russell Tiffin', 'I Shivram': 'Ivaturi Shivram', 'TH Wijewardene': 'Tyron Wijewardene', 'AL Hill': 'Tony Hill', 'Subroto Das': 'Subroto Das', 
                         'K Srinivasan': 'Krishnamachari Srinivasan', 'AM Saheba': 'Amiesh Saheba', 'RM Deshpande': 'Rajesh Deshpande', 'K Bharatan': 'Krishnamachari Bharatan', 'YC Barde': 'Yeshwant Barde', 
                         'UV Gandhe': 'Ulhas Gandhe', 'SD Ranade': 'Shashank Ranade'}

In [ ]:
player_names = {'TM Head': 'Travis Head', 'DT Christian': 'Daniel Christian', 'DS Kulkarni': 'Dhawal Kulkarni', 'CJ Anderson': 'Corey Anderson', 'AR Bawne': 'Ankit Bawne', 'SP Jackson': 'Sheldon Jackson', 'MJ Guptill': 'Martin Guptill', 'Anureet Singh': 'Anureet Singh', 'DJ Thornely': 'Dominic Thornely', 'PR Shah': 'Pinal Shah', 'AM Nayar': 'Abhishek Nayar', 'SM Pollock': 'Shaun Pollock', 'AS Yadav': 'Arjun Yadav', 'MA Khote': 'Musavir Khote', 'DA Warner': 'David Warner', 'S Dhawan': 'Shikhar Dhawan', 'MC Henriques': 'Moises Henriques', 'Yuvraj Singh': 'Yuvraj Singh', 'DJ Hooda': 'Deepak Hooda', 'BCJ Cutting': 'Ben Cutting', 'CH Gayle': 'Chris Gayle', 'Mandeep Singh': 'Mandeep Singh', 'KM Jadhav': 'Kedar Jadhav', 'SR Watson': 'Shane Watson', 'Sachin Baby': 'Sachin Baby', 'STR Binny': 'Stuart Binny', 'S Aravind': 'Sreenath Aravind', 'YS Chahal': 'Yuzvendra Chahal', 'TS Mills': 'Tymal Mills', 'A Choudhary': 'Aniket Choudhary', 'PA Patel': 'Parthiv Patel', 'JC Buttler': 'Jos Buttler', 'RG Sharma': 'Rohit Sharma', 'N Rana': 'Nitish Rana', 'AT Rayudu': 'Ambati Rayudu', 'KH Pandya': 'Krunal Pandya', 'KA Pollard': 'Kieron Pollard', 'HH Pandya': 'Hardik Pandya', 'TG Southee': 'Tim Southee', 'AM Rahane': 'Ajinkya Rahane', 'MA Agarwal': 'Mayank Agarwal', 'SPD Smith': 'Steve Smith', 'BA Stokes': 'Ben Stokes', 
               'MS Dhoni': 'Mahendra Singh Dhoni', 'JJ Roy': 'Jason Roy', 'BB McCullum': 'Brendon McCullum', 'SK Raina': 'Suresh Raina', 'AJ Finch': 'Aaron Finch', 'KD Karthik': 'Dinesh Karthik', 'G Gambhir': 'Gautam Gambhir', 'CA Lynn': 'Chris Lynn', 'MK Tiwary': 'Manoj Tiwary', 'HM Amla': 'Hashim Amla', 'M Vohra': 'Manan Vohra', 'WP Saha': 'Wriddhiman Saha', 'AR Patel': 'Axar Patel', 'GJ Maxwell': 'Glenn Maxwell', 'DA Miller': 'David Miller', 'Vishnu Vinod': 'Vishnu Vinod', 'Iqbal Abdulla': 'Iqbal Abdulla', 'P Negi': 'Pawan Negi', 'AP Tare': 'Aditya Tare', 'SW Billings': 'Sam Billings', 'KK Nair': 'Karun Nair', 'SV Samson': 'Sanju Samson', 'RR Pant': 'Rishabh Pant', 'CH Morris': 'Chris Morris', 'CR Brathwaite': 'Carlos Brathwaite', 'PJ Cummins': 'Pat Cummins', 'A Mishra': 'Amit Mishra', 'S Nadeem': 'Shahbaz Nadeem', 'Z Khan': 'Zaheer Khan', 'DR Smith': 'Dwayne Smith', 'P Kumar': 'Praveen Kumar', 'Basil Thampi': 'Basil Thampi', 'RV Uthappa': 'Robin Uthappa', 'MK Pandey': 'Manish Pandey', 'YK Pathan': 'Yusuf Pathan', 'SA Yadav': 'Suryakumar Yadav', 'CR Woakes': 'Chris Woakes', 'SP Narine': 'Sunil Narine', 'Harbhajan Singh': 'Harbhajan Singh', 'AB de Villiers': 'AB de Villiers', 'F du Plessis': 'Faf du Plessis', 'RA Tripathi': 'Rahul Tripathi', 'R Bhatia': 'Rajat Bhatia', 'DL Chahar': 'Deepak Chahar', 'A Zampa': 'Adam Zampa', 
               'AB Dinda': 'Ashoke Dinda', 'Imran Tahir': 'Imran Tahir', 'NV Ojha': 'Naman Ojha', 'V Shankar': 'Vijay Shankar', 'Rashid Khan': 'Rashid Khan', 'B Kumar': 'Bhuvneshwar Kumar', 'MP Stoinis': 'Marcus Stoinis', 'MM Sharma': 'Mohit Sharma', 'VR Aaron': 'Varun Aaron', 'V Kohli': 'Virat Kohli', 'MJ McClenaghan': 'Mitchell McClenaghan', 'Ankit Sharma': 'Ankit Sharma', 'SN Thakur': 'Shardul Thakur', 'RD Chahar': 'Rahul Chahar', 'LH Ferguson': 'Lockie Ferguson', 'C de Grandhomme': 'Colin de Grandhomme', 'Bipul Sharma': 'Bipul Sharma', 'SS Iyer': 'Shreyas Iyer', 'EJG Morgan': 'Eoin Morgan', 'KC Cariappa': 'KC Cariappa', 'Sandeep Sharma': 'Sandeep Sharma', 'Ishan Kishan': 'Ishan Kishan', 'JD Unadkat': 'Jaydev Unadkat', 'AF Milne': 'Adam Milne', 'S Badree': 'Samuel Badree', 'AD Mathews': 'Angelo Mathews', 'Mohammed Shami': 'Mohammed Shami', 'Mohammad Nabi': 'Mohammad Nabi', 'I Sharma': 'Ishant Sharma', 'RA Jadeja': 'Ravindra Jadeja', 'AJ Tye': 'Andrew Tye', 'KS Williamson': 'Kane Williamson', 'SE Marsh': 'Shaun Marsh', 'Shakib Al Hasan': 'Shakib Al Hasan', 'JP Faulkner': 'James Faulkner', 'MG Johnson': 'Mitchell Johnson', 'K Rabada': 'Kagiso Rabada', 'AD Nath': 'Akshdeep Nath', 'NM Coulter-Nile': 'Nathan Coulter-Nile', 'Kuldeep Yadav': 'Kuldeep Yadav', 'UT Yadav': 'Umesh Yadav', 'Washington Sundar': 'Washington Sundar', 
               'KV Sharma': 'Karn Sharma', 'DM Bravo': 'Darren Bravo', 'IK Pathan': 'Irfan Pathan', 'Ankit Soni': 'Ankit Soni', 'JJ Bumrah': 'Jasprit Bumrah', 'SL Malinga': 'Lasith Malinga', 'PJ Sangwan': 'Pradeep Sangwan', 'S Kaul': 'Siddarth Kaul', 'LMP Simmons': 'Lendl Simmons', 'MN Samuels': 'Marlon Samuels', 'Swapnil Singh': 'Swapnil Singh', 'R Tewatia': 'Rahul Tewatia', 'MM Patel': 'Munaf Patel', 'SS Tiwary': 'Saurabh Tiwary', 'TA Boult': 'Trent Boult', 'CJ Jordan': 'Chris Jordan', 'IR Jaggi': 'Ishank Jaggi', 'PP Chawla': 'Piyush Chawla', 'AS Rajpoot': 'Ankit Rajpoot', 'SC Ganguly': 'Sourav Ganguly', 'RT Ponting': 'Ricky Ponting', 'DJ Hussey': 'David Hussey', 'Mohammad Hafeez': 'Mohammad Hafeez', 'R Dravid': 'Rahul Dravid', 'W Jaffer': 'Wasim Jaffer', 'JH Kallis': 'Jacques Kallis', 'CL White': 'Cameron White', 'MV Boucher': 'Mark Boucher', 'B Akhil': 'Balachandra Akhil', 'AA Noffke': 'Ashley Noffke', 'SB Joshi': 'Sunil Joshi', 'ML Hayden': 'Matthew Hayden', 'MEK Hussey': 'Michael Hussey', 'JDP Oram': 'Jacob Oram', 'S Badrinath': 'Subramaniam Badrinath', 'K Goel': 'Karan Goel', 'JR Hopes': 'James Hopes', 'KC Sangakkara': 'Kumar Sangakkara', 'SM Katich': 'Simon Katich', 'T Kohli': 'Taruwar Kohli', 'M Kaif': 'Mohammad Kaif', 'DS Lehmann': 'Darren Lehmann', 'M Rawat': 'Mahesh Rawat', 'D Salunkhe': 'Dinesh Salunkhe', 'SK Warne': 'Shane Warne', 
               'SK Trivedi': 'Siddharth Trivedi', 'V Sehwag': 'Virender Sehwag', 'L Ronchi': 'Luke Ronchi', 'ST Jayasuriya': 'Sanath Jayasuriya', 'S Chanderpaul': 'Shivnarine Chanderpaul', 'LRPL Taylor': 'Ross Taylor', 'AC Gilchrist': 'Adam Gilchrist', 'Y Venugopal Rao': 'Venugopal Rao', 'VVS Laxman': 'VVS Laxman', 'A Symonds': 'Andrew Symonds', 'SB Styris': 'Scott Styris', 'SB Bangar': 'Sanjay Bangar', 'WPUJC Vaas': 'Chaminda Vaas', 'RP Singh': 'RP Singh', 'LR Shukla': 'Laxmi Shukla', 'DPMD Jayawardene': 'Mahela Jayawardene', 'S Sohal': 'Sunny Sohal', 'B Lee': 'Brett Lee', 'WA Mota': 'Wilkin Mota', 'Kamran Akmal': 'Kamran Akmal', 'Shahid Afridi': 'Shahid Afridi', 'DJ Bravo': 'Dwayne Bravo', 'A Nehra': 'Ashish Nehra', 'GC Smith': 'Graeme Smith', 'Pankaj Singh': 'Pankaj Singh', 'RR Sarwan': 'Ramnaresh Sarwan', 'S Sreesanth': 'S Sreesanth', 'VRV Singh': 'Vikram Singh', 'R Vinay Kumar': 'Vinay Kumar', 'AB Agarkar': 'Ajit Agarkar', 'M Kartik': 'Murali Kartik', 'Shoaib Malik': 'Shoaib Malik', 'MF Maharoof': 'Farveez Maharoof', 'VY Mahesh': 'Yo Mahesh', 'TM Srivastava': 'Tanmay Srivastava', 'B Chipli': 'Bharat Chipli', 'DW Steyn': 'Dale Steyn', 'DB Das': 'Debabrata Das', 'HH Gibbs': 'Herschelle Gibbs', 'DNT Zoysa': 'Nuwan Zoysa', 'D Kalyankrishna': 'Doddapaneni Kalyankrishna', 'SA Asnodkar': 'Swapnil Asnodkar', 'Sohail Tanvir': 'Sohail Tanvir', 
               'Salman Butt': 'Salman Butt', 'BJ Hodge': 'Brad Hodge', 'Umar Gul': 'Umar Gul', 'SP Fleming': 'Stephen Fleming', 'S Vidyut': 'Vidyut Sivaramakrishnan', 'JA Morkel': 'Albie Morkel', 'LPC Silva': 'Chamara Silva', 'DB Ravi Teja': 'Dwaraka Ravi Teja', 'Misbah-ul-Haq': 'Misbah-ul-Haq', 'YV Takawale': 'Yogesh Takawale', 'RR Raje': 'Rohan Raje', 'Mohammad Asif': 'Mohammad Asif', 'GD McGrath': 'Glenn McGrath', 'Joginder Sharma': 'Joginder Sharma', 'MS Gony': 'Manpreet Gony', 'M Muralitharan': 'Muttiah Muralitharan', 'M Ntini': 'Makhaya Ntini', 'DT Patil': 'Devraj Patil ', 'A Kumble': 'Anil Kumble', 'S Anirudha': 'Anirudha Srikkanth', 'CK Kapugedera': 'Chamara Kapugedera', 'A Chopra': 'Aakash Chopra', 'T Taibu': 'Tatenda Taibu', 'J Arunkumar': 'Jagadeesh Arunkumar', 'PP Ojha': 'Pragyan Ojha', 'SP Goswami': 'Shreevats Goswami', 'SR Tendulkar': 'Sachin Tendulkar', 'LA Pomersbach': 'Luke Pomersbach', 'Younis Khan': 'Younis Khan', 'PM Sarvesh Kumar': 'Sarvesh Kumar', 'DP Vijaykumar': 'Paidikalva Vijaykumar', 'Shoaib Akhtar': 'Shoaib Akhtar', 'Abdur Razzak': 'Abdur Razzak', 'H Das': 'Halhadar Das', 'SD Chitnis': 'Siddharth Chitnis', 'CRD Fernando': 'Dilhara Fernando', 'L Balaji': 'Lakshmipathy Balaji', 'A Mukund': 'Abhinav Mukund', 'RR Powar': 'Ramesh Powar', 'JP Duminy': 'JP Duminy', 'A Flintoff': 'Andrew Flintoff', 'T Thushara': 'Thilan Thushara', 'JD Ryder': 'Jesse Ryder', 
               'KP Pietersen': 'Kevin Pietersen', 'T Henderson': 'Tyron Henderson', 'Kamran Khan': 'Kamran Khan', 'RS Bopara': 'Ravi Bopara', 'R Bishnoi': 'Rajesh Bishnoi', 'FH Edwards': 'Fidel Edwards', 'PC Valthaty': 'Paul Valthaty', 'RJ Quiney': 'Rob Quiney', 'AS Raut': 'Abhishek Raut', 'Yashpal Singh': 'Yashpal Singh', 'M Manhas': 'Mithun Manhas', 'AN Ghosh': 'Arindam Ghosh', 'BAW Mendis': 'Ajantha Mendis', 'DL Vettori': 'Daniel Vettori', 'MN van Wyk': 'Morné van Wyk', 'RE van der Merwe': 'Roelof van der Merwe', 'TL Suman': 'Tirumalasetti Suman', 'Shoaib Ahmed': 'Shoaib Ahmed', 'GR Napier': 'Graham Napier', 'KP Appanna': 'K P Appanna', 'LA Carseldine': 'Lee Carseldine', 'SM Harwood': 'Shane Harwood', 'M Vijay': 'Murali Vijay', 'SB Jakati': 'Shadab Jakati', 'RJ Harris': 'Ryan Harris', 'D du Preez': 'Dillon du Preez', 'M Morkel': 'Morne Morkel', 'J Botha': 'Johan Botha', 'C Nanda': 'Chetanya Nanda', 'Mashrafe Mortaza': 'Mashrafe Mortaza', 'A Singh': 'Anureet Singh', 'GJ Bailey': 'George Bailey', 'AB McDonald': 'Andrew McDonald', 'Y Nagar': 'Yogesh Nagar', 'SS Shaikh': 'Shoaib Shaikh', 'R Ashwin': 'Ravichandran Ashwin', 'Mohammad Ashraful': 'Mohammad Ashraful', 'CA Pujara': 'Cheteshwar Pujara', 'OA Shah': 'Owais Shah', 'Anirudh Singh': 'Anirudh Singh', 'Jaskaran Singh': 'Jaskaran Singh', 'R Sathish': 'Rajagopal Sathish', 'R McLaren': 'Ryan McLaren', 'AA Jhunjhunwala': 'Abhishek Jhunjhunwala', 
               'P Dogra': 'Paras Dogra', 'A Uniyal': 'Amit Uniyal', 'MS Bisla': 'Manvinder Bisla', 'YA Abdulla': 'Yusuf Abdulla', 'JM Kemp': 'Justin Kemp', 'S Tyagi': 'Sudeep Tyagi', 'RS Gavaskar': 'Rohan Gavaskar', 'SE Bond': 'Shane Bond', 'S Ladda': 'Sarabjit Ladda', 'DP Nannes': 'Dirk Nannes', 'MJ Lumb': 'Michael Lumb', 'DR Martyn': 'Damien Martyn', 'S Narwal': 'Sumit Narwal', 'AB Barath': 'Adrian Barath', 'FY Fazal': 'Faiz Fazal', 'AC Voges': 'Adam Voges', 'MD Mishra': 'Mohnish Mishra', 'KB Arun Karthik': 'Arun Karthik', 'KAJ Roach': 'Kemar Roach', 'PD Collingwood': 'Paul Collingwood', 'CK Langeveldt': 'Charl Langeveldt', 'VS Malik': 'Vikramjeet Malik', 'A Mithun': 'Abhimanyu Mithun', 'AN Ahmed': 'Abu Nechim', 'RS Sodhi': 'Reetinder Singh Sodhi', 'DE Bollinger': 'Doug Bollinger', 'S Sriram': 'Sridharan Sriram', 'B Sumanth': 'Bodapati Sumanth', 'C Madan': 'Chandan Madan', 'AG Paunikar': 'Amit Paunikar', 'MR Marsh': 'Mitchell Marsh', 'Harmeet Singh': 'Harmeet Singh', 'RV Gomez': 'Raiphi Gomez', 'AUK Pathan': 'Asad Pathan', 'UBT Chand': 'Unmukt Chand', 'DJ Jacobs': 'Davy Jacobs', 'Sunny Singh': 'Sunny Singh', 'NJ Rimmington': 'Nathan Rimmington', 'AL Menaria': 'Ashok Menaria', 'WD Parnell': 'Wayne Parnell', 'JJ van der Wath': 'Johan van der Wath', 'R Ninan': 'Ryan Ninan', 'MS Wade': 'Matthew Wade', 'TD Paine': 'Tim Paine', 'SB Wagh': 'Shrikant Wagh', 'AC Thomas': 'Alfonso Thomas', 
               'JEC Franklin': 'James Franklin', 'DH Yagnik': 'Dishant Yagnik', 'S Randiv': 'Suraj Randiv', 'BJ Haddin': 'Brad Haddin', 'NLTC Perera': 'Thisara Perera', 'NL McCullum': 'Nathan McCullum', 'JE Taylor': 'Jerome Taylor', 'J Syed Mohammad': 'Syed Mohammad', 'RN ten Doeschate': 'Ryan ten Doeschate', 'TR Birt': 'Travis Birt', 'Harpreet Singh': 'Harpreet Singh Bhatia', 'M Klinger': 'Michael Klinger', 'AC Blizzard': 'Aiden Blizzard', 'I Malhotra': 'Ishan Malhotra', 'L Ablish': 'Love Ablish', 'CJ Ferguson': 'Callum Ferguson', 'AA Chavan': 'Ankeet Chavan', 'ND Doshi': 'Nayan Doshi', 'Y Gnaneswara Rao': 'Gnaneshwara Rao', 'S Rana': 'Sachin Rana', 'BA Bhatt': 'Bhargav Bhatt', 'RE Levi': 'Richard Levi', 'KK Cooper': 'Kevon Cooper', 'HV Patel': 'Harshal Patel', 'DAJ Bracewell': 'Doug Bracewell', 'GB Hogg': 'Brad Hogg', 'RR Bhatkal': 'Raju Bhatkal', 'CJ McKay': 'Clint McKay', 'N Saini': 'Navdeep Saini', 'Azhar Mahmood': 'Azhar Mahmood', 'RJ Peterson': 'Robin Peterson', 'KMDN Kulasekara': 'Nuwan Kulasekara', 'A Ashish Reddy': 'Ashish Reddy', 'V Pratap Singh': 'Veer Pratap Singh', 'BB Samantray': 'Biplab Samantray', 'MJ Clarke': 'Michael Clarke', 'Gurkeerat Singh': 'Gurkeerat Singh', 'AP Majumdar': 'Anustup Majumdar', 'PA Reddy': 'Akshath Reddy', 'K Upadhyay': 'Krishnakant Upadhyay', 'P Awana': 'Parvinder Awana', 'AD Russell': 'Andre Russell', 'A Chandila': 'Ajit Chandila', 'Sunny Gupta': 'Sunny Gupta', 
               'MC Juneja': 'Manpreet Juneja', 'GH Vihari': 'Hanuma Vihari', 'MDKJ Perera': 'Kusal Perera', 'R Shukla': 'Rahul Shukla', 'B Laughlin': 'Ben Laughlin', 'BMAJ Mendis': 'Jeevan Mendis', 'R Rampaul': 'Ravi Rampaul', 'BJ Rohrer': 'Ben Rohrer', 'KL Rahul': 'KL Rahul', 'Q de Kock': 'Quinton de Kock', 'R Dhawan': 'Rishi Dhawan', 'LJ Wright': 'Luke Wright', 'IC Pandey': 'Ishwar Pandey', 'CM Gautam': 'CM Gautam', 'DJG Sammy': 'Daren Sammy', 'KW Richardson': 'Kane Richardson', 'UA Birla': 'Udit Birla', 'Parvez Rasool': 'Parvez Rasool', 'PV Tambe': 'Pravin Tambe', 'NJ Maddinson': 'Nic Maddinson', 'JDS Neesham': 'James Neesham', 'MA Starc': 'Mitchell Starc', 'BR Dunk': 'Ben Dunk', 'RR Rossouw': 'Rilee Rossouw', 'Shivam Sharma': 'Shivam Sharma', 'VH Zol': 'Vijay Zol', 'BE Hendricks': 'Beuran Hendricks', 'S Gopal': 'Shreyas Gopal', 'M de Lange': 'Marchant de Lange', 'JO Holder': 'Jason Holder', 'Karanveer Singh': 'Karanveer Singh', 'SA Abbott': 'Sean Abbott', 'J Suchith': 'Jagadeesha Suchith', 'RG More': 'Ronit More', 'D Wiese': 'David Wiese', 'SN Khan': 'Sarfaraz Khan', 'DJ Muthuswami': 'Domnic Joseph', 'C Munro': 'Colin Munro', 'P Sahu': 'Pardeep Sahu', 'KJ Abbott': 'Kyle Abbott', 'M Ashwin': 'Murugan Ashwin', 'NS Naik': 'Nikhil Naik', 'PSP Handscomb': 'Peter Handscomb', 'J Yadav': 'Jayant Yadav', 'UT Khawaja': 'Usman Khawaja', 'F Behardien': 'Farhaan Behardien', 'BB Sran': 'Barinder Sran', 'S Kaushik': 'Shivil Kaushik', 
               'ER Dwivedi': 'Eklavya Dwivedi', 'E Lewis': 'Evin Lewis', 'M Wood': 'Mark Wood','K Gowtham': 'Krishnappa Gowtham', 'T Curran': 'Tom Curran', 'M Markande': 'Mayank Markande', 'B Stanlake': 'Billy Stanlake', 'M Ur Rahman': 'Mujeeb Ur Rahman', 'A Dananjaya': 'Akila Dananjaya', 'L Plunkett': 'Liam Plunkett', 'Mustafizur Rahman': 'Mustafizur Rahman', 'A Hales': 'Alex Hales', 'M Lomror': 'Mahipal Lomror', 'D Shorey': 'Dhruv Shorey', 'P Krishna': 'Prasidh Krishna', 'P Chopra': 'Prashant Chopra', 'S Dube': 'Shivam Dube', 'R Salam': 'Rasikh Salam', 'N Pooran': 'Nicholas Pooran', 'N Naik': 'Nikhil Naik', 'H Vihari': 'Hanuma Vihari', 'P R Barman': 'Prayas Ray Barman', 'H Viljoen': 'Hardus Viljoen', 'Avesh Khan': 'Avesh Khan', 'S Lamichhane': 'Sandeep Lamichhane', 'S Sharma': 'Sandeep Sharma', 'H Gurney': 'Harry Gurney', 'SD Lad': 'Siddhesh Lad', 'A Joseph': 'Alzarri Joseph', 'R Parag': 'Riyan Parag', 'M Santner': 'Mitchell Santner', 'J Denly': 'Joe Denly', 'L Livingstone': 'Liam Livingstone', 'K Ahmed': 'Khaleel Ahmed', 'A Turner': 'Ashton Turner', 'H Brar': 'Harpreet Brar', 'S Rutherford': 'Sherfane Rutherford', 
               'P Raj': 'Prithvi Raj', 'AR Bawne': 'Ankit Bawne', 'SP Jackson': 'Sheldon Jackson', 'MJ Guptill': 'Martin Guptill', 'Anureet Singh': 'Anureet Singh', 'DJ Thornely': 'Dominic Thornely', 'AM Nayar': 'Abhishek Nayar', 'SM Pollock': 'Shaun Pollock', 'TM Dilshan': 'Tillakaratne Dilshan', 'AD Mascarenhas': 'Dimitri Mascarenhas', 'Niraj Patel': 'Niraj Patel', 'A Nel': 'André Nel', 'J Theron': 'Rusty Theron', 'SJ Srivastava': 'Shalabh Srivastava', 'SW Tait': 'Shaun Tait', 'C Ganapathy': 'Chandrasekar Ganapathy', 'P Parameswaran': 'Prasanth Parameswaran', 'CA Ingram': 'Colin Ingram', 'TP Sudhindra': 'TP Sudhindra', 'BW Hilfenhaus': 'Benjamin Hilfenhaus', 'Mohammed Siraj': 'Mohammed Siraj', 'H Klaasen': 'Heinrich Klaasen', 'J Archer': 'Jofra Archer', 'R Bhui': 'Ricky Bhui', 'Tejas Baroka': 'Tejas Baroka', 'SS Agarwal': 'Shubham Agarwal', 'MJ Henry': 'Matt Henry', 'P Amarnath': 'Palani Amarnath', 'B Geeves': 'Brett Geeves', 'Gagandeep Singh': 'Gagandeep Singh', 'AM Salvi': 'Aavishkar Salvi', 'RR Bose': 'Ranadeb Bose', 'SS Sarkar': 'Soumya Sarkar', 'AA Kazi': 'Abrar Kazi', 'Anand Rajan': 'Anand Rajan', 'P Prasanth': 'Padmanabhan Prasanth', 'SS Mundhe': 'Shrikant Mundhe', 'RW Price': 'Ray Price', 'Harmeet Singh': 'Harmeet Singh', 'P Suyal': 'Pawan Suyal', 'MG Neser': 'Michael Neser', 'K Santokie': 'Krishmar Santokie', 'JW Hastings': 'John Hastings', 'GS Sandhu': 'Gurinder Sandhu', 
               'T Shamsi': 'Tabraiz Shamsi', 'SM Boland': 'Scott Boland', 'K Khejroliya': 'Kulwant Khejroliya', 'L Ngidi': 'Lungi Ngidi', 'KM Asif': 'KM Asif', 'D Willey': 'David Willey', 'L Ferguson': 'Lockie Ferguson', 'V Chakravarthy': 'Varun Chakravarthy', 'J Behrendorff': 'Jason Behrendorff', 'S Kuggeleijn': 'Scott Kuggeleijn', 'S Midhun': 'Sudhesan Midhun', 'O Thomas': 'Oshane Thomas', 'A Roy': 'Anukul Roy', 'S Warrier': 'Sandeep Warrier', 'U Kaul': 'Uday Kaul', 'TM Dilshan': 'Tillakaratne Dilshan', 'AD Mascarenhas': 'Dimitri Mascarenhas', 'Niraj Patel': 'Niraj Patel', 'VS Yeligati': 'Vikrant Yeligati', 'AA Bilakhia': 'Azhar Bilakhia', 'J Theron': 'Juan Theron', 'SJ Srivastava': 'Shalabh Srivastava', 'R Sharma': 'Rahul Sharma', 'SW Tait': 'Shaun Tait', 'AP Dole': 'Aditya Dole', 'AG Murtaza': 'Ali Murtuza', 'CA Ingram': 'Colin Ingram', 'P Parameswaran': 'Prashanth Parameswaran', 'DJ Harris': 'Daniel Harris', 'MSM Senanayake': 'Sachithra Senanayake', 'X Thalaivan Sargunam': 'Thalaivan Sargunam', 'Mohammed Siraj': 'Mohammed Siraj', 'H Klaasen': 'Heinrich Klaasen', 'R Bhui': 'Ricky Bhui', 'NB Singh': 'Nathu Singh', 'RA Shaikh': 'Rahil Shaikh', 'MB Parmar': 'Mohnsinh Parmar', 'AT Carey': 'Alex Carey', 'Y Prithvi Raj': 'Prithvi Raj', 'CJ Dala': 'Junior Dala', 'SO Hetmyer': 'Shimron Hetmyer', 'Mujeeb Ur Rahman': 'Mujeeb Ur Rahman', 'MK Lomror': 'Mahipal Lomror', 'I Udana': 'Isuru Udana', 'YBK Jaiswal': 'Yashasvi Jaiswal', 
               'TK Curran': 'Tom Curran', 'Shubman Gill': 'Shubman Gill', 'AD Hales': 'Alex Hales', 'Rasikh Salam': 'Rasikh Salam', 'M Prasidh Krishna': 'Prasidh Krishna', 'CV Varun': 'Varun Chakravarthy', 'P Simran Singh': 'Prabhsimran Singh', 'RK Bhui': 'Ricky Bhui', 'Abdul Samad': 'Abdul Samad', 'DR Sams': 'Daniel Sams', 'JP Behrendorff': 'Jason Behrendorff', 'AS Roy': 'Anukul Roy', 'JM Bairstow': 'Jonny Bairstow', 'N Jagadeesan': 'Narayan Jagadeesan', 'SS Cottrell': 'Sheldon Cottrell', 'MM Ali': 'Moeen Ali', 'Ravi Bishnoi': 'Ravi Bishnoi', 'LE Plunkett': 'Liam Plunkett', 'IS Sodhi': 'Ish Sodhi', 'Kartik Tyagi': 'Kartik Tyagi', 'AS Joseph': 'Alzarri Joseph', 'Shivam Mavi': 'Shivam Mavi', 'JR Philippe': 'Josh Philippe', 'SE Rutherford': 'Sherfane Rutherford', 'KL Nagarkoti': 'Kamlesh Nagarkoti', 'Monu Kumar': 'Monu Kumar', 'SC Kuggeleijn': 'Scott Kuggeleijn', 'KMA Paul': 'Keemo Paul', 'LS Livingstone': 'Liam Livingstone', 'MA Wood': 'Mark Wood', 'SM Curran': 'Sam Curran', 'P Dubey': 'Praveen Dubey', 'GC Viljoen': 'Hardus Viljoen', 'MJ Santner': 'Mitchell Santner', 'RD Gaikwad': 'Ruturaj Gaikwad', 'CJ Green': 'Chris Green', 'DR Shorey': 'Dhruv Shorey', 'JL Denly': 'Joe Denly', 'JL Pattinson': 'James Pattinson', 'JC Archer': 'Jofra Archer', 'T Banton': 'Tom Banton', 'S Sandeep Warrier': 'Sandeep Warrier', 'JR Hazlewood': 'Josh Hazlewood', 'Shahbaz Ahmed': 'Shahbaz Ahmed', 'AJ Turner': 'Ashton Turner', 'Arshdeep Singh': 'Arshdeep Singh', 'KK Ahmed': 'Khaleel Ahmed', 
               'JPR Scantlebury-Searles': 'Javon Searles', 'HF Gurney': 'Harry Gurney', 'DJ Willey': 'David Willey', 'Abhishek Sharma': 'Abhishek Sharma', 'A Nortje': 'Anrich Nortje', 'Harpreet Brar': 'Harpreet Brar', 'TU Deshpande': 'Tushar Deshpande', 'D Padikkal': 'Devdutt Padikkal', 'PP Shaw': 'Prithvi Shaw', 'SMSM Senanayake': 'Sachithra Senanayake', 'RK Singh': 'Rinku Singh', 'NA Saini': 'Navdeep Saini', 'DJM Short': "D'Arcy Short", 'PK Garg': 'Priyam Garg', 'P Ray Barman': 'Prayas Barman'}

Let us write our final function :)

In [ ]:
def change_name_format(matches, player_names, umpire_names):
    matches.replace(player_names, inplace = True)
    matches.replace(umpire_names, inplace = True)

In [ ]:
change_name_format(matches, player_names, umpire_names)
matches.head()

Now, it's more readbale and easy to interpret, right? 

So, we have completed the Feature Engineering of `ipl-matches` dataset. Let us save it with .csv extension.

In [ ]:
matches.to_csv('matches.csv')      # file path

# Deliveries Datasets
Let us quickly apply some functions which we have defined earlier in this notebook to preprocess `deliveries` dataset.

## Loading the Datasets

In [ ]:
deliveries = pd.read_csv('../input/ipl-complete-dataset-20082020/IPL Ball-by-Ball 2008-2020.csv', index_col='id')
deliveries.head()

## Data Preprocessing
We just need to replace few team names as we did earlier and change the player's names format. Let us call the functions which we have defined earlier to do the same.

In [ ]:
columns_to_change = ['batting_team', 'bowling_team']
replace_team_names(deliveries, columns_to_change)
change_name_format(deliveries, player_names, umpire_names)
deliveries.head()

It's ready now. Let us just export it as csv file.

In [ ]:
deliveries.to_csv('deliveries.csv')

You may consider upvoting this notebook if you like my work. It really encourage me to do more :)